# Creates and simulates a circuit equivalent to a Bell inequality test

Referance:
https://cirq.readthedocs.io/en/stable/examples.html#bell-inequality

In [1]:
import numpy as np
import cirq

In [2]:
def make_bell_test_circuit():
    alice = cirq.GridQubit(0, 0)
    bob = cirq.GridQubit(1, 0)
    alice_referee = cirq.GridQubit(0, 1)
    bob_referee = cirq.GridQubit(1, 1)

    circuit = cirq.Circuit()

    # Prepare shared entangled state.
    circuit.append([
        cirq.H(alice),
        cirq.CNOT(alice, bob),
        cirq.X(alice)**-0.25,
    ])

    # Referees flip coins.
    circuit.append([
        cirq.H(alice_referee),
        cirq.H(bob_referee),
    ], strategy=cirq.InsertStrategy.NEW_THEN_INLINE)

    # Players do a sqrt(X) based on their referee's coin.
    circuit.append([
        cirq.CNOT(alice_referee, alice)**0.5,
        cirq.CNOT(bob_referee, bob)**0.5,
    ], strategy=cirq.InsertStrategy.NEW_THEN_INLINE)

    # Then results are recorded.
    circuit.append([
        cirq.measure(alice, key='a'),
        cirq.measure(bob, key='b'),
        cirq.measure(alice_referee, key='x'),
        cirq.measure(bob_referee, key='y'),
    ], strategy=cirq.InsertStrategy.NEW_THEN_INLINE)

    return circuit

In [3]:
# Create circuit.
circuit = make_bell_test_circuit()
print('Circuit:')
print(circuit)

Circuit:
(0, 0): ───H───@───X^-0.25───────X───────M('a')───
               │                 │
(0, 1): ───────┼─────────────H───@^0.5───M('x')───
               │
(1, 0): ───────X─────────────────X───────M('b')───
                                 │
(1, 1): ─────────────────────H───@^0.5───M('y')───


In [4]:
# Run simulations.
repetitions = 10000
print('Simulating {} repetitions...'.format(repetitions))
result = cirq.Simulator().run(program=circuit, repetitions=repetitions)

Simulating 10000 repetitions...


In [5]:
# Collect results.
a = np.array(result.measurements['a'][:, 0])
b = np.array(result.measurements['b'][:, 0])
x = np.array(result.measurements['x'][:, 0])
y = np.array(result.measurements['y'][:, 0])
outcomes = a ^ b == x & y
win_percent = len([e for e in outcomes if e]) * 100 / repetitions

In [6]:
def bitstring(bits):
    return ''.join('1' if e else '_' for e in bits)

In [6]:
# Print data.
print()
print('Results')
# print('a:', bitstring(a))
# print('b:', bitstring(b))
# print('x:', bitstring(x))
# print('y:', bitstring(y))
# print('(a XOR b) == (x AND y):\n  ', bitstring(outcomes))
print('Win rate: {}%'.format(win_percent))


Results
Win rate: 85.11%


In [7]:
np.cos(np.pi/8)**2

0.8535533905932737